In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
from pyproj import Transformer, Geod
import pandas as pd
import sys
import ast
import numpy as np
import matplotlib.pyplot as plt
if ".." not in sys.path: sys.path.insert(0,"..")
from wildfire.Reader import Reader as WFReader

In [8]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring


def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point

In [9]:
def fire_within_time_range(feature, start_year=1963, end_year=2023):
    fire_year = int(feature['attributes']['Fire_Year'])
    return (start_year <= fire_year) & (fire_year <= end_year)

def fire_within_distance_range(feature, place, max_distance=1250):
    """_summary_

    Parameters
    ----------
    feature : dictionary
        _description_
    place : list of floats
        (latitude, longitude)
    """
    ring_data = feature['geometry']['rings'][0]
    distance = shortest_distance_from_place_to_fire_perimeter(place,ring_data)[0]
    return distance, distance <= max_distance

In [2]:
fire_assignments = pd.read_excel("../Wildfire Cities Assignments.xlsx")
assignment = fire_assignments[fire_assignments['LastName'] == "Yip"]
assignment

,LastName,FirstName,State,City,Population,County
57,Yip,Evan Yuen Fei,Montana,Missoula,76955,Missoula


In [3]:
# Geocoordinates of Missoula, Montana
city_coordinates = [46.8721, -113.9940]

In [4]:

FINAL_DATA_FILENAME = "../GeoJSON Exports/USGS_Wildland_Fire_Combined_Dataset.json"
wfreader = WFReader(FINAL_DATA_FILENAME)

In [ ]:
valid_feature_list = list()
failed_feature_list = list()
distances = list()
feature_count = 0
valid_feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result 
wfreader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_count += 1
    fire_year = int(feature['attributes']['Fire_Year'])
    # Only add fires within the last 60 years
    try:
        if fire_within_time_range(feature):
            print("year", int(feature['attributes']['Fire_Year']))
            distance, in_range = fire_within_distance_range(feature, city_coordinates)
            if in_range:
                valid_feature_list.append(feature)
                distances.append(distance)
                valid_feature_count += 1
                print(valid_feature_count)
    except Exception as E:
        print(E)
        failed_feature_list.append(feature)
    feature = wfreader.next()
#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {valid_feature_count} features")
#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(valid_feature_list)} features")

In [ ]:
feature_attributes = [feature['attributes'] for feature in valid_feature_list]
feature_geometries = [Polygon(feature['geometry']['rings'][0]) for feature in valid_feature_list]
feature_df = gpd.GeoDataFrame(feature_attributes)
feature_df['geometry'] = feature_geometries
feature_df.set_geometry(col='geometry', inplace=True)
# Saving the file to geojson format
feature_df.to_file('./valid_wildfire_features2.geojson', driver='GeoJSON')

# Save distances to file
pd.DataFrame({'distances':distances}).to_csv('./distances.csv', index=False)

# Write failed features
failed_feature_attributes = [feature['attributes'] for feature in failed_feature_list]
failed_feature_geometries = [feature['geometry'] for feature in failed_feature_list]
failed_feature_df = pd.DataFrame(failed_feature_attributes)
failed_feature_df['geometry'] = failed_feature_geometries
failed_feature_df.to_csv('./failed_wildfire_features2.csv')

### Loading the extracted, cleaned geojson and csv files

In [ ]:
valid_distances = pd.read_csv('./distances.csv')
failed_features = pd.read_csv('./failed_wildfire_features2.csv')
gdf = gpd.read_file('./valid_wildfire_features2.geojson', engine='pyogrio')
gdf['distances'] = valid_distances['distances']

In [ ]:
fires = []
distances = []
geometries = []
for i, feature in enumerate(failed_features['geometry']):
    ring_data = ast.literal_eval(feature)['curveRings'][0]
    # print(ring_data[0])
    clean_ring_data = list()
    for j, val in enumerate(ring_data):
        if isinstance(val, list):
            clean_ring_data.append(val)
    distance = shortest_distance_from_place_to_fire_perimeter(city_coordinates, clean_ring_data)
    # Checking distances
    fires.append(distance[0] <= 1250)
    if distance[0] <= 1250:
        distances.append(distance[0])
        geometries.append(Polygon(clean_ring_data))

In [ ]:
close_fires = failed_features.iloc[fires, :]
close_fires['geometry'] = geometries
close_fires['distances'] = distances

In [ ]:
# Concatenating the initial pass fires and the failed fires together
fire_gdf = pd.concat([gdf, close_fires])
fire_gdf.to_file('../data_intermediate/wildfire_combine.geojson', driver='GeoJSON')


### Step 1: Create fire smoke estimates
What are the estimated smoke impacts on your assigned city for the last 60 years.

I will first filter the dataset down to only `Wildfire`, `Likely Wildfire` and `Unknown - Likely Wildfire` fire types because we are interested in smoke estimates of **wildfires** not prescribed fires.

In [ ]:
fire_gdf[['Assigned_Fire_Type', 'OBJECTID']].groupby('Assigned_Fire_Type').count()

,OBJECTID
Assigned_Fire_Type,
Likely Wildfire,4682
Prescribed Fire,20636
Unknown - Likely Prescribed Fire,1656
Unknown - Likely Wildfire,157
Wildfire,53587


I will define my smoke estimate for that year as the following:

$\textnormal{smoke estimate} = \textnormal{number of fires that year}  * \textnormal{acres burned} * \textnormal{distance from city}$

$$SE(i) = x = \begin{cases}
   \sum_{j=0}^{n}(1*\textnormal{acres}_{ij} / \textnormal{distance}_{ij} ) &\text{;if }  wildfire = 1\\
   \sum_{j=0}^{n}(0.25*\textnormal{acres}_{ij} / \textnormal{distance}_{ij} ) &\text{;if } wildfire = 0
\end{cases}

$$

Where:
- $n$ is the number of wildfires
- year $i$. 
- acres = sigmoid((acres - mean(acres))/stdev(acres))
- distance = sigmoid((distance - mean(distance))/stdev(distance))


In [ ]:
features = ['OBJECTID', 'Fire_Year', 'GIS_Acres', 'distances', 'Assigned_Fire_Type']
fire_features = fire_gdf[features]
fire_features['fire_type'] = np.where(fire_features['Assigned_Fire_Type'].isin(['Wildfire', 'Unknown - Likely Wildfire', 'Likely Wildfire']), 1, 0.25)

In [ ]:
def sigmoid(x):
    return 1/(1 + np.exp(-x)) 

def StandardScaler(df, columns):
    for col in columns:
        mean = df[col].mean()
        sd = df[col].std()
        df[f"{col}_scaled"] = sigmoid((df[col] - mean) / sd)
    return df

def smoke_estimate(acres, distance, fire_type):
    return acres / distance * fire_type

fire_features.columns = ['id', 'year', 'acres', 'distance', 'fire_type', 'type_val']
fire_scaled = StandardScaler(fire_features, ['acres', 'distance'])

smoke_estimate_df = fire_scaled.assign(smoke_estimate = lambda row: smoke_estimate(row['acres_scaled'], row['distance_scaled'], row['type_val']))
smoke_estimate_df = smoke_estimate_df[['year', 'smoke_estimate']].groupby('year').sum().reset_index()

# Saving smoke estimate to dataframe
smoke_estimate_df.to_csv('../data_intermediate/smoke_estimate.csv', index=False)